In [1]:
import os
os.environ["SPS_HOME"] = "/Users/fpetri/packages/fsps" 

In [2]:
import numpy as np
from speculator import SpectrumPCA
from speculator import  Photulator

In [4]:
import speculator

In [ ]:
speculator.train_photulator_stack()

In [3]:
photo_emulator  = Photulator(n_parameters=16, filters=)

SyntaxError: invalid syntax (3112674364.py, line 1)